pycaret based mechine learning , integrated multi algorithm to predict morphology features extracted by cellprofiler<br>
kernel:'pycaret'

In [66]:
import os 
import pandas as pd

In [67]:
ucec = pd.read_csv('/data/dk/ucec/cf_out/Image.csv',delimiter=',')
ucec.select_dtypes(include='object')
columns_to_keep = [col for col in ucec.columns if col == 'FileName_myimage' or ucec[col].dtype != 'object']
ucec2 = ucec[columns_to_keep]
ucec2.loc[:,'case']  = ucec2['FileName_myimage'].str.split('_tile').str[0]
ucec3 = ucec2.groupby('case').mean(numeric_only = True)
refer_data = pd.read_csv('/data/dk/ucec/DataSheet_2.csv')
ucec3 = ucec3[refer_data.drop(columns=['case']).columns]
ucec3 = ucec3.reset_index()
ucec3['case_submitter_id'] = ucec3['case'].str.extract(r'^(.*?-.*?-.*?)-',expand = False)
ucec_meta = pd.read_csv('/data/dk/ucec/clinical.tsv', delimiter='\t')[['case_submitter_id','figo_stage']]
data_ucec = ucec3.merge(ucec_meta,on='case_submitter_id',how='inner')
replacements = {
    r'Stage IV.*': "Stage_IV",
    r'Stage III.*': "Stage_III",
    r'Stage II.*': "Stage_II",
    r'Stage I.*': "Stage_I"
}
for pattern ,replace in replacements.items():
    data_ucec['figo_stage'] = data_ucec['figo_stage'].str.replace(pattern,replace)
data_ucec.drop(columns=['case','case_submitter_id'],inplace=True)

In [68]:
tma = pd.read_csv('/data/dk/endometrium/cf_out/Image.csv',delimiter=',')
tma.select_dtypes(include='object')
columns_to_keep = [col for col in tma.columns if col == 'FileName_myimage' or tma[col].dtype != 'object']
filtered_df = tma[columns_to_keep]
filtered_df.loc[:,'case']  = filtered_df['FileName_myimage'].str.split('_tile').str[0]
filtered_df2 = filtered_df.groupby('case').mean(numeric_only = True)
# refer_data = pd.read_csv('/data/dk/ucec/cf_out/DataSheet_2.csv')
tma3 = filtered_df2[refer_data.drop(columns=['case']).columns]
endo_meta = pd.read_csv('/data/dk/ucec/endo_meta.csv', delimiter=',').iloc[:,[0,15]]
endo_meta.rename(columns={'序号': 'case_submitter_id', 'TNM': 'figo_stage'}, inplace=True)
tma3 = tma3.reset_index()
tma3['case_submitter_id'] = tma3['case'].str.extract('([A-Z])(\d+)').apply(lambda x: f"{x[0]}{x[1].zfill(2)}", axis=1)
data_tma3 = tma3.merge(endo_meta,on='case_submitter_id',how='inner')
replacements = {
    r'T1.*': "Stage_I",
    r'T4.*': "Stage_IV",
    r'T3.*': "Stage_III"
}
for pattern ,replace in replacements.items():
    data_tma3['figo_stage'] = data_tma3['figo_stage'].str.replace(pattern,replace)
data_tma3.drop(columns=['case','case_submitter_id'],inplace=True)

In [69]:
data_tma3.drop(list(data_tma3.filter(regex = 'Threshold')), axis = 1, inplace = True)
data_ucec.drop(list(data_ucec.filter(regex = 'Threshold')), axis = 1, inplace = True)


In [70]:

import pycaret
from pycaret.classification import *


In [71]:
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

In [72]:
# init setup on exp
setup(data_ucec, target = 'figo_stage', train_size = 0.8, normalize = False, 
            test_data = data_tma3,session_id = 123,index=False, 
            fix_imbalance = True)

,Description,Value
0,Session id,123
1,Target,figo_stage
2,Target type,Multiclass
3,Target mapping,"Stage_I: 0, Stage_II: 1, Stage_III: 2, Stage_IV: 3"
4,Original data shape,"(1180, 556)"
5,Transformed data shape,"(2904, 556)"
6,Transformed train set shape,"(2856, 556)"
7,Transformed test set shape,"(48, 556)"
8,Numeric features,555
9,Preprocess,True


In [73]:
best = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.6307,0.5000,0.6307,0.3978,0.4879,0.0000,0.0000,0.2370
lightgbm,Light Gradient Boosting Machine,0.5636,0.5631,0.5636,0.5017,0.5207,0.0858,0.0906,0.2990
rf,Random Forest Classifier,0.5513,0.5440,0.5513,0.4830,0.5015,0.0422,0.0376,0.2510
catboost,CatBoost Classifier,0.5495,0.5528,0.5495,0.4917,0.5061,0.0630,0.0649,0.2750
xgboost,Extreme Gradient Boosting,0.5406,0.5585,0.5406,0.4992,0.5100,0.0763,0.0786,0.2490
et,Extra Trees Classifier,0.5337,0.5492,0.5337,0.4824,0.4904,0.0270,0.0284,0.2540
gbc,Gradient Boosting Classifier,0.5017,0.5186,0.5017,0.4766,0.4822,0.0389,0.0405,0.2520
ada,Ada Boost Classifier,0.4294,0.5175,0.4294,0.4800,0.4434,0.0424,0.0436,0.2420
dt,Decision Tree Classifier,0.4258,0.5374,0.4258,0.5021,0.4496,0.0720,0.0767,0.3620
ridge,Ridge Classifier,0.3975,0.0000,0.3975,0.5342,0.4304,0.0902,0.0989,0.3600


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

In [74]:
dt = create_model('lr')
tuned_dt = tune_model(dt, optimize = 'AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3684,0.5612,0.3684,0.5504,0.4259,0.0843,0.0926
1,0.3158,0.5963,0.3158,0.6386,0.3785,0.1313,0.1651
2,0.2301,0.4933,0.2301,0.4030,0.2714,-0.0482,-0.0559
3,0.3894,0.5745,0.3894,0.4991,0.4334,0.0354,0.0369
4,0.3982,0.7194,0.3982,0.5788,0.4315,0.1562,0.1766
5,0.3628,0.5411,0.3628,0.5154,0.4142,0.0568,0.0621
6,0.4248,0.6066,0.4248,0.5114,0.4592,0.0776,0.0807
7,0.4602,0.6560,0.4602,0.5887,0.5102,0.1558,0.1642
8,0.2743,0.5037,0.2743,0.4104,0.2799,0.0595,0.0722


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3509,0.5518,0.3509,0.5680,0.4150,0.0834,0.0937
1,0.3158,0.6058,0.3158,0.6360,0.3845,0.1220,0.1510
2,0.2832,0.5095,0.2832,0.5004,0.3297,0.0419,0.0497
3,0.4071,0.5541,0.4071,0.5064,0.4477,0.0478,0.0495
4,0.4690,0.7475,0.4690,0.6457,0.5149,0.2295,0.2514
5,0.3805,0.5352,0.3805,0.5351,0.4399,0.0612,0.0658
6,0.4602,0.6345,0.4602,0.5529,0.4988,0.1260,0.1305
7,0.4779,0.6486,0.4779,0.6359,0.5368,0.2020,0.2144
8,0.2743,0.4935,0.2743,0.4095,0.2656,0.0771,0.0960


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [92]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [127]:
X_train = data_ucec.drop('figo_stage', axis=1).values
X_test = data_tma3.drop('figo_stage', axis=1).values
y_train = data_ucec['figo_stage'].values
y_test = data_tma3['figo_stage'].values


In [130]:
from sklearn.linear_model import LogisticRegression

y_score = tuned_dt.predict_proba(X_test)

In [117]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  

(566, 4)

In [ ]:
from itertools import cycle

fig, ax = plt.subplots(figsize=(6, 6))

plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.2f})",
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label=f"macro-average ROC curve (AUC = {roc_auc['macro']:.2f})",
    color="navy",
    linestyle=":",
    linewidth=4,
)

colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for class_id, color in zip(range(n_classes), colors):
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_score[:, class_id],
        name=f"ROC curve for {target_names[class_id]}",
        color=color,
        ax=ax
    )

plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Extension of Receiver Operating Characteristic\nto One-vs-Rest multiclass")
plt.legend()
plt.show()

In [115]:
ktarget_names = data_ucec['figo_stage'].unique()
target_names


array(['Stage_III', 'Stage_I', 'Stage_II', 'Stage_IV'], dtype=object)